# Define two functions to calibrate

In [1]:
# Define a function

import pandas as pd
import numpy as np
import os

def seg_travel_cali(file_name):
    
    f = open('Calibration_' + str(file_name[:-4]) + '_Summary' + '.txt', "a") # open a text file to record prints
    print('Schedule Calibration:', file_name[:-4])
    print()
    print('Schedule Calibration:', file_name[:-4], file=f)

    df = pd.read_csv(file_name)

    seg_names = df['Segment'].tolist()
    seg_names = list(set(seg_names))
    print(len(seg_names), 'segments found', seg_names)
    print()
    print(len(seg_names), 'segments found', seg_names, file=f)

    # 50 EAST
    # Create a new column and calculate the order of the segment

    df['SEG'] = df.apply(seg_order, axis=1)

    # Sort by trip start time, Date and segment order
    df = df.sort_values(['TripStartTime', 'Date', 'SEG'], ascending=[True, True, True])

    # Create a unique column 'TripStartTime' + 'SEG_ORDER'
    df['TRIP_START_TIME_SEG'] = df['TripStartTime'] + ' ' + df['SEG']

    # Check how many trips start times
    trip_start_times = df['TripStartTime'].tolist()
    trip_start_times = list(set(trip_start_times))
    # trip_start_times

    print('The direction has', len(trip_start_times), 'different trips during the day')
    print()

    # convert the travel time columns into int
    df = df.replace(',','', regex=True) # emove , in numbers
    df['Actual Running Time'] = df['Actual Running Time'].astype(int) 
    df['Scheduled Running Time'] = df['Scheduled Running Time'].astype(int) 


    #  Modify outliers
    df['ACTUAL_RUNNING_T_NO_OUTL'] = df['Actual Running Time'].astype(int)  # create a no outlier column

    TRIP_START_TIME_SEG_ls = df['TRIP_START_TIME_SEG'].tolist()
    TRIP_START_TIME_SEG_ls = list(set(TRIP_START_TIME_SEG_ls)) # remove duplicate, return a list of TRIP_START_TIME_SEG
    # print(TRIP_START_TIME_SEG_ls, len(TRIP_START_TIME_SEG_ls))

    # Select a subset 
    for element in TRIP_START_TIME_SEG_ls:
        df_sub = df[df["TRIP_START_TIME_SEG"] == element] # select the time of a 'start time + segment' 
        actual_time_ls = df_sub['Actual Running Time'].tolist()
        actual_time_ls.sort()
    #     print(actual_time_ls)
        q1, q3= np.percentile(actual_time_ls,[25,75])
    #     print(q1, q3)
        iqr = q3 - q1
        lower_bound = q1 -(1.5 * iqr) 
        upper_bound = q3 +(1.5 * iqr) # find outliers using interquartile bound
    #     print(lower_bound, upper_bound)

        # modify the outliers if they are out of the interquartile bound range
    #     df.loc[(df.TRIP_START_TIME_SEG==element) & (df.ACTUAL_RUNNING_T_NO_OUTL>upper_bound), 'BB'] = upper_bound
    #     df.loc[(df.TRIP_START_TIME_SEG==element) & (df.ACTUAL_RUNNING_T_NO_OUTL<lower_bound), 'BBB'] = lower_bound
        df.loc[(df.TRIP_START_TIME_SEG==element) & (df.ACTUAL_RUNNING_T_NO_OUTL>upper_bound), 'ACTUAL_RUNNING_T_NO_OUTL'] = upper_bound
        df.loc[(df.TRIP_START_TIME_SEG==element) & (df.ACTUAL_RUNNING_T_NO_OUTL<lower_bound), 'ACTUAL_RUNNING_T_NO_OUTL'] = lower_bound

    # Summarize by TRIP_START_TIME_SEG
    df_optm_time = df.groupby("TRIP_START_TIME_SEG")["ACTUAL_RUNNING_T_NO_OUTL"].quantile(.90)
    df_optm_time.to_csv('df_optm_time_temporary_file.csv') 
    df_optm_time = pd.read_csv('df_optm_time_temporary_file.csv') # reset the index
    df_optm_time['OPTM_SCHED_TIME'] = df_optm_time['ACTUAL_RUNNING_T_NO_OUTL'] # copy the needed average to a new column
    df_optm_time = df_optm_time.drop(['ACTUAL_RUNNING_T_NO_OUTL'], axis = 1) # delete the other column
    # print(df_optm_time)

    # Join tables
    df_optm_time = pd.merge(df, df_optm_time, on='TRIP_START_TIME_SEG')

    df_sched_time = df.groupby("TRIP_START_TIME_SEG")["Scheduled Running Time"].mean()
    
    df_sched_time.to_csv('df_sched_time_temporary_file.csv') 
    df_sched_time = pd.read_csv('df_sched_time_temporary_file.csv') # reset the index
    df_sched_time['CURRENT_SCHED_TIME'] = df_sched_time['Scheduled Running Time'] # copy the data to a new column
    df_sched_time = df_sched_time.drop(['Scheduled Running Time'], axis = 1)

    df_optm_time = pd.merge(df_optm_time, df_sched_time, on='TRIP_START_TIME_SEG') # merge the scheduled travel time

    # df_optm_time.to_csv('df_optm_time.csv')  

    df_optm_time['PRECALI_DIFF'] = df_optm_time['Scheduled Running Time'] - df_optm_time['Actual Running Time']
    df_optm_time['POSTCALI_DIFF'] = df_optm_time['OPTM_SCHED_TIME'] - df_optm_time['Actual Running Time']
    
    # Sort by trip start time, Date and segment order
#     df_optm_time = df_optm_time.sort_values(['TripStartTime', 'Date', 'SEG'], ascending=[True, True, True])
    df_optm_time = df_optm_time.sort_values(['TripStartTime', 'TRIP_START_TIME_SEG'], ascending=[True, True])

    df_optm_time.to_csv('Calibration_' + str(file_name[:-4]) + '_Details' + '.csv')  

    s = df_optm_time.sum(axis=0) # provide a summary stats
    # print(s)
    # print(s['OPTM_SCHED_TIME'])

    # Calculate performance improve before and after Calibration
    print('Total scheduled hours before calibration from the input dates:', round(s['CURRENT_SCHED_TIME'] / 3600, 2), file=f)
    print('Total scheduled hours after calibration from the input dates:', round(s['OPTM_SCHED_TIME'] / 3600, 2), file=f)
    print('Variance (off in seconds) before calibration:', round(df_optm_time.var()['PRECALI_DIFF'], 2), file=f)
    print('Variance (off in seconds) after calibration:', round(df_optm_time.var()['POSTCALI_DIFF'], 2), file=f)

    print('Total scheduled hours before calibration from the input dates:', round(s['CURRENT_SCHED_TIME'] / 3600, 2))
    print('Total scheduled hours after calibration from the input dates:', round(s['OPTM_SCHED_TIME'] / 3600, 2))
    print()
    print('Variance (off in seconds) before calibration:', round(df_optm_time.var()['PRECALI_DIFF'], 2))
    print('Variance (off in seconds) after calibration:', round(df_optm_time.var()['POSTCALI_DIFF'], 2))
    print()

    # print('SD (off in seconds) before calibration:', round(df_optm_time.std()['PRECALI_DIFF'], 2))
    # print('SD (off in seconds) after calibration:', round(df_optm_time.std()['POSTCALI_DIFF'], 2))
    # print()

    e1 = df_optm_time['PRECALI_DIFF'].tolist()
    i= len(e1)
    j = 0
    for t in e1:
        if t > 0:
            j = j+1

    print('Segment trips have sufficient time to travel before calibration:', round(j/i, 3),'    ', j, ':', i, file=f)
    print('Segment trips have sufficient time to travel before calibration:', round(j/i, 3),'    ', j, ':', i)

    e2 = df_optm_time['POSTCALI_DIFF'].tolist()
    i= len(e2)
    j = 0
    for t in e2:
        if t > 0:
            j = j+1

    print('Segment trips have sufficient time to travel after calibration:', round(j/i, 3), '    ', j, ':', i, file=f)
    print('Segment trips have sufficient time to travel after calibration:', round(j/i, 3), '    ', j, ':', i)
    print()

    l1 = df_optm_time['PRECALI_DIFF'].tolist()
    i= len(l1)
    j = 0
    for t in l1:
        if t <= -0:
            j = j+1

    print('Segment trips without sufficient time to travel before calibration:', round(j/i, 3), '    ', j, ':', i, file=f)
    print('Segment trips without sufficient time to travel before calibration:', round(j/i, 3), '    ', j, ':', i)

    l2 = df_optm_time['POSTCALI_DIFF'].tolist()
    i= len(l2)
    j = 0
    for t in l2:
        if t <= -0:
            j = j+1

    print('Segment trips without sufficient time to travel after calibration:', round(j/i, 3), '    ', j, ':', i, file=f)
    print('Segment trips without sufficient time to travel after calibration:', round(j/i, 3), '    ', j, ':', i)
    print()

#     o1 = df_optm_time['PRECALI_DIFF'].tolist()
#     i= len(o1)
#     j = 0
#     for t in o1:
#         if t <= -300 or t > 60:
#             j = j+1

#     print('Segment on time performance before calibration for the input dates:', round(1- j/i, 3), '    ', j, ':', i, file=f)
#     print('Segment on time performance before calibration for the input dates:', round(1- j/i, 3), '    ', j, ':', i)

#     o2 = df_optm_time['POSTCALI_DIFF'].tolist()
#     i= len(o2)
#     j = 0
#     for t in o2:
#         if t <= -300 or t > 60:
#             j = j+1
#     print('Segment on time performance after calibration for the input date:',  round(1-j/i, 3), '    ', j, ':', i, file=f)
#     print('Segment on time performance after calibration for the input date:',  round(1-j/i, 3), '    ', j, ':', i)

    f.close()  

    # simplify the export to service planner

    simple_df_opt = df_optm_time.drop(['Date', 'Actual Running Time','SEG','PRECALI_DIFF', 
                                       'POSTCALI_DIFF', 'Scheduled Running Time', 'ACTUAL_RUNNING_T_NO_OUTL'], axis=1)

    simple_df_opt = simple_df_opt.round({"OPTM_SCHED_TIME":0}) # around up the 

    simple_df_opt = simple_df_opt.drop_duplicates()# remove duplicates 

    simple_df_opt.to_csv('Calibration_' + file_name[:-4] + '_Brief' + '.csv')  

#     print(simple_df_opt)
    
    os.remove('df_sched_time_temporary_file.csv') # remove temporary file
    os.remove('df_optm_time_temporary_file.csv') # remove temporary file
    
    print()
    print('Done')
    print()
    print('-----------------------------------------')

## EAST

In [2]:
# Test the function Weekdays EAST

def seg_order(row):
    if row['Segment'] == 'bearid - sanout':
        order = 'E1'
    elif row['Segment'] == 'sanout - heapar':
        order = 'E2'
    elif row['Segment'] == 'heapar - winsum':
        order = 'E3'
    elif row['Segment'] == 'winsum - btwrsh':
        order = 'E4'
    elif row['Segment'] == 'btwrsh - dantre':
        order = 'E5'
    elif row['Segment'] == 'dantre - rsw':
        order = 'E6'
#    elif row['Segment'] == 'rptc - marpal':
#        order = 'E'
#     elif row['Segment'] == 'estcre - estbow': 
#         order = 'E5'
#     elif row['Segment'] == 'dantre - rsw':
#         order = 'E6'
    else:
        order = 'E'
    return order

file_name = '50 EAST Weekdays - Dec - Mar.csv'
seg_travel_cali(file_name)

Schedule Calibration: 50 EAST Weekdays - Dec - Mar

6 segments found ['bearid - sanout', 'heapar - winsum', 'sanout - heapar', 'dantre - rsw', 'btwrsh - dantre', 'winsum - btwrsh']

The direction has 14 different trips during the day

Total scheduled hours before calibration from the input dates: 1163.4
Total scheduled hours after calibration from the input dates: 1261.24

Variance (off in seconds) before calibration: 15670.07
Variance (off in seconds) after calibration: 13678.16

Segment trips have sufficient time to travel before calibration: 0.706      4458 : 6317
Segment trips have sufficient time to travel after calibration: 0.893      5639 : 6317

Segment trips without sufficient time to travel before calibration: 0.294      1859 : 6317
Segment trips without sufficient time to travel after calibration: 0.107      678 : 6317


Done

-----------------------------------------


C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:111: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('Variance (off in seconds) before calibration:', round(df_optm_time.var()['PRECALI_DIFF'], 2), file=f)
C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:112: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('Variance (off in seconds) after calibration:', round(df_optm_time.var()['POSTCALI_DIFF'], 2), file=f)
C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:117: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise Typ

In [3]:
# Test the function Saturdays EAST

def seg_order(row):
    if row['Segment'] == 'bearid - sanout':
        order = 'E1'
    elif row['Segment'] == 'sanout - heapar':
        order = 'E2'
    elif row['Segment'] == 'heapar - winsum':
        order = 'E3'
    elif row['Segment'] == 'winsum - btwrsh':
        order = 'E4'
    elif row['Segment'] == 'btwrsh - dantre':
        order = 'E5'
    elif row['Segment'] == 'dantre - rsw':
        order = 'E6'
#    elif row['Segment'] == 'rptc - marpal':
#        order = 'E'
#     elif row['Segment'] == 'estcre - estbow': 
#         order = 'E5'
#     elif row['Segment'] == 'dantre - rsw':
#         order = 'E6'
    else:
        order = 'E'
    return order

file_name = '50 EAST Saturdays - Dec - Mar.csv'
seg_travel_cali(file_name)

Schedule Calibration: 50 EAST Saturdays - Dec - Mar

6 segments found ['bearid - sanout', 'heapar - winsum', 'sanout - heapar', 'dantre - rsw', 'btwrsh - dantre', 'winsum - btwrsh']

The direction has 14 different trips during the day

Total scheduled hours before calibration from the input dates: 217.45
Total scheduled hours after calibration from the input dates: 226.05

Variance (off in seconds) before calibration: 12177.97
Variance (off in seconds) after calibration: 9322.9

Segment trips have sufficient time to travel before calibration: 0.751      892 : 1188
Segment trips have sufficient time to travel after calibration: 0.869      1032 : 1188

Segment trips without sufficient time to travel before calibration: 0.249      296 : 1188
Segment trips without sufficient time to travel after calibration: 0.131      156 : 1188


Done

-----------------------------------------


C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:111: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('Variance (off in seconds) before calibration:', round(df_optm_time.var()['PRECALI_DIFF'], 2), file=f)
C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:112: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('Variance (off in seconds) after calibration:', round(df_optm_time.var()['POSTCALI_DIFF'], 2), file=f)
C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:117: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise Typ

In [4]:
# Test the function EAST Sundays

def seg_order(row):
    if row['Segment'] == 'bearid - sanout':
        order = 'E1'
    elif row['Segment'] == 'sanout - heapar':
        order = 'E2'
    elif row['Segment'] == 'heapar - winsum':
        order = 'E3'
    elif row['Segment'] == 'winsum - btwrsh':
        order = 'E4'
    elif row['Segment'] == 'btwrsh - dantre':
        order = 'E5'
    elif row['Segment'] == 'dantre - rsw':
        order = 'E6'
#    elif row['Segment'] == 'rptc - marpal':
#        order = 'E'
#     elif row['Segment'] == 'estcre - estbow': 
#         order = 'E5'
#     elif row['Segment'] == 'dantre - rsw':
#         order = 'E6'
    else:
        order = 'E'
    return order

file_name = '50 EAST Sundays - Dec - Mar.csv'
seg_travel_cali(file_name)

Schedule Calibration: 50 EAST Sundays - Dec - Mar

6 segments found ['bearid - sanout', 'heapar - winsum', 'sanout - heapar', 'dantre - rsw', 'btwrsh - dantre', 'winsum - btwrsh']

The direction has 6 different trips during the day

Total scheduled hours before calibration from the input dates: 92.58
Total scheduled hours after calibration from the input dates: 92.46

Variance (off in seconds) before calibration: 15582.21
Variance (off in seconds) after calibration: 6910.38

Segment trips have sufficient time to travel before calibration: 0.767      385 : 502
Segment trips have sufficient time to travel after calibration: 0.873      438 : 502

Segment trips without sufficient time to travel before calibration: 0.233      117 : 502
Segment trips without sufficient time to travel after calibration: 0.127      64 : 502


Done

-----------------------------------------


C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:111: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('Variance (off in seconds) before calibration:', round(df_optm_time.var()['PRECALI_DIFF'], 2), file=f)
C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:112: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('Variance (off in seconds) after calibration:', round(df_optm_time.var()['POSTCALI_DIFF'], 2), file=f)
C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:117: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise Typ

In [5]:
# Test the function Weekdays-Saturdays EAST

def seg_order(row):
    if row['Segment'] == 'bearid - sanout':
        order = 'E1'
    elif row['Segment'] == 'sanout - heapar':
        order = 'E2'
    elif row['Segment'] == 'heapar - winsum':
        order = 'E3'
    elif row['Segment'] == 'winsum - btwrsh':
        order = 'E4'
    elif row['Segment'] == 'btwrsh - dantre':
        order = 'E5'
    elif row['Segment'] == 'dantre - rsw':
        order = 'E6'
#    elif row['Segment'] == 'rptc - marpal':
#        order = 'E'
#     elif row['Segment'] == 'estcre - estbow': 
#         order = 'E5'
#     elif row['Segment'] == 'dantre - rsw':
#         order = 'E6'
    else:
        order = 'E'
    return order

file_name = '50 EAST Weekdays-Saturday - Dec - Mar.csv'
seg_travel_cali(file_name)

Schedule Calibration: 50 EAST Weekdays-Saturday - Dec - Mar

6 segments found ['bearid - sanout', 'heapar - winsum', 'sanout - heapar', 'dantre - rsw', 'btwrsh - dantre', 'winsum - btwrsh']

The direction has 14 different trips during the day

Total scheduled hours before calibration from the input dates: 1380.85
Total scheduled hours after calibration from the input dates: 1494.01

Variance (off in seconds) before calibration: 15167.3
Variance (off in seconds) after calibration: 13653.45

Segment trips have sufficient time to travel before calibration: 0.713      5350 : 7505
Segment trips have sufficient time to travel after calibration: 0.892      6697 : 7505

Segment trips without sufficient time to travel before calibration: 0.287      2155 : 7505
Segment trips without sufficient time to travel after calibration: 0.108      808 : 7505


Done

-----------------------------------------


C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:111: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('Variance (off in seconds) before calibration:', round(df_optm_time.var()['PRECALI_DIFF'], 2), file=f)
C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:112: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('Variance (off in seconds) after calibration:', round(df_optm_time.var()['POSTCALI_DIFF'], 2), file=f)
C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:117: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise Typ

## WEST

In [6]:
# Define a function

import pandas as pd
import numpy as np
import os

def seg_travel_cali(file_name):
    
    f = open('Calibration_' + str(file_name[:-4]) + '_Summary' + '.txt', "a") # open a text file to record prints
    print('Schedule Calibration:', file_name[:-4])
    print()
    print('Schedule Calibration:', file_name[:-4], file=f)

    df = pd.read_csv(file_name)

    seg_names = df['Segment'].tolist()
    seg_names = list(set(seg_names))
    print(len(seg_names), 'segments found', seg_names)
    print()
    print(len(seg_names), 'segments found', seg_names, file=f)

    # 50 EAST
    # Create a new column and calculate the order of the segment

    df['SEG'] = df.apply(seg_order, axis=1)

    # Sort by trip start time, Date and segment order
    df = df.sort_values(['TripStartTime', 'Date', 'SEG'], ascending=[True, True, True])

    # Create a unique column 'TripStartTime' + 'SEG_ORDER'
    df['TRIP_START_TIME_SEG'] = df['TripStartTime'] + ' ' + df['SEG']

    # Check how many trips start times
    trip_start_times = df['TripStartTime'].tolist()
    trip_start_times = list(set(trip_start_times))
    # trip_start_times

    print('The direction has', len(trip_start_times), 'different trips during the day')
    print()

    # convert the travel time columns into int
    df = df.replace(',','', regex=True) # emove , in numbers
    df['Actual Running Time'] = df['Actual Running Time'].astype(int) 
    df['Scheduled Running Time'] = df['Scheduled Running Time'].astype(int) 


    #  Modify outliers
    df['ACTUAL_RUNNING_T_NO_OUTL'] = df['Actual Running Time'].astype(int)  # create a no outlier column

    TRIP_START_TIME_SEG_ls = df['TRIP_START_TIME_SEG'].tolist()
    TRIP_START_TIME_SEG_ls = list(set(TRIP_START_TIME_SEG_ls)) # remove duplicate, return a list of TRIP_START_TIME_SEG
    # print(TRIP_START_TIME_SEG_ls, len(TRIP_START_TIME_SEG_ls))

    # Select a subset 
    for element in TRIP_START_TIME_SEG_ls:
        df_sub = df[df["TRIP_START_TIME_SEG"] == element] # select the time of a 'start time + segment' 
        actual_time_ls = df_sub['Actual Running Time'].tolist()
        actual_time_ls.sort()
    #     print(actual_time_ls)
        q1, q3= np.percentile(actual_time_ls,[25,75])
    #     print(q1, q3)
        iqr = q3 - q1
        lower_bound = q1 -(1.5 * iqr) 
        upper_bound = q3 +(1.5 * iqr) # find outliers using interquartile bound
    #     print(lower_bound, upper_bound)

        # modify the outliers if they are out of the interquartile bound range
    #     df.loc[(df.TRIP_START_TIME_SEG==element) & (df.ACTUAL_RUNNING_T_NO_OUTL>upper_bound), 'BB'] = upper_bound
    #     df.loc[(df.TRIP_START_TIME_SEG==element) & (df.ACTUAL_RUNNING_T_NO_OUTL<lower_bound), 'BBB'] = lower_bound
        df.loc[(df.TRIP_START_TIME_SEG==element) & (df.ACTUAL_RUNNING_T_NO_OUTL>upper_bound), 'ACTUAL_RUNNING_T_NO_OUTL'] = upper_bound
        df.loc[(df.TRIP_START_TIME_SEG==element) & (df.ACTUAL_RUNNING_T_NO_OUTL<lower_bound), 'ACTUAL_RUNNING_T_NO_OUTL'] = lower_bound

    # Summarize by TRIP_START_TIME_SEG
    df_optm_time = df.groupby("TRIP_START_TIME_SEG")["ACTUAL_RUNNING_T_NO_OUTL"].quantile(.90)
    df_optm_time.to_csv('df_optm_time_temporary_file.csv') 
    df_optm_time = pd.read_csv('df_optm_time_temporary_file.csv') # reset the index
    df_optm_time['OPTM_SCHED_TIME'] = df_optm_time['ACTUAL_RUNNING_T_NO_OUTL'] # copy the needed average to a new column
    df_optm_time = df_optm_time.drop(['ACTUAL_RUNNING_T_NO_OUTL'], axis = 1) # delete the other column
    # print(df_optm_time)

    # Join tables
    df_optm_time = pd.merge(df, df_optm_time, on='TRIP_START_TIME_SEG')

    df_sched_time = df.groupby("TRIP_START_TIME_SEG")["Scheduled Running Time"].mean()
    
    df_sched_time.to_csv('df_sched_time_temporary_file.csv') 
    df_sched_time = pd.read_csv('df_sched_time_temporary_file.csv') # reset the index
    df_sched_time['CURRENT_SCHED_TIME'] = df_sched_time['Scheduled Running Time'] # copy the data to a new column
    df_sched_time = df_sched_time.drop(['Scheduled Running Time'], axis = 1)

    df_optm_time = pd.merge(df_optm_time, df_sched_time, on='TRIP_START_TIME_SEG') # merge the scheduled travel time

    # df_optm_time.to_csv('df_optm_time.csv')  

    df_optm_time['PRECALI_DIFF'] = df_optm_time['Scheduled Running Time'] - df_optm_time['Actual Running Time']
    df_optm_time['POSTCALI_DIFF'] = df_optm_time['OPTM_SCHED_TIME'] - df_optm_time['Actual Running Time']
    
    # Sort by trip start time, Date and segment order
#     df_optm_time = df_optm_time.sort_values(['TripStartTime', 'Date', 'SEG'], ascending=[True, True, True])
    df_optm_time = df_optm_time.sort_values(['TripStartTime', 'TRIP_START_TIME_SEG'], ascending=[True, True])

    df_optm_time.to_csv('Calibration_' + str(file_name[:-4]) + '_Details' + '.csv')  

    s = df_optm_time.sum(axis=0) # provide a summary stats
    # print(s)
    # print(s['OPTM_SCHED_TIME'])

    # Calculate performance improve before and after Calibration
    print('Total scheduled hours before calibration from the input dates:', round(s['CURRENT_SCHED_TIME'] / 3600, 2), file=f)
    print('Total scheduled hours after calibration from the input dates:', round(s['OPTM_SCHED_TIME'] / 3600, 2), file=f)
    print('Variance (off in seconds) before calibration:', round(df_optm_time.var()['PRECALI_DIFF'], 2), file=f)
    print('Variance (off in seconds) after calibration:', round(df_optm_time.var()['POSTCALI_DIFF'], 2), file=f)

    print('Total scheduled hours before calibration from the input dates:', round(s['CURRENT_SCHED_TIME'] / 3600, 2))
    print('Total scheduled hours after calibration from the input dates:', round(s['OPTM_SCHED_TIME'] / 3600, 2))
    print()
    print('Variance (off in seconds) before calibration:', round(df_optm_time.var()['PRECALI_DIFF'], 2))
    print('Variance (off in seconds) after calibration:', round(df_optm_time.var()['POSTCALI_DIFF'], 2))
    print()

    # print('SD (off in seconds) before calibration:', round(df_optm_time.std()['PRECALI_DIFF'], 2))
    # print('SD (off in seconds) after calibration:', round(df_optm_time.std()['POSTCALI_DIFF'], 2))
    # print()

    e1 = df_optm_time['PRECALI_DIFF'].tolist()
    i= len(e1)
    j = 0
    for t in e1:
        if t > 0:
            j = j+1

    print('Segment trips have sufficient time to travel before calibration:', round(j/i, 3),'    ', j, ':', i, file=f)
    print('Segment trips have sufficient time to travel before calibration:', round(j/i, 3),'    ', j, ':', i)

    e2 = df_optm_time['POSTCALI_DIFF'].tolist()
    i= len(e2)
    j = 0
    for t in e2:
        if t > 0:
            j = j+1

    print('Segment trips have sufficient time to travel after calibration:', round(j/i, 3), '    ', j, ':', i, file=f)
    print('Segment trips have sufficient time to travel after calibration:', round(j/i, 3), '    ', j, ':', i)
    print()

    l1 = df_optm_time['PRECALI_DIFF'].tolist()
    i= len(l1)
    j = 0
    for t in l1:
        if t <= -0:
            j = j+1

    print('Segment trips without sufficient time to travel before calibration:', round(j/i, 3), '    ', j, ':', i, file=f)
    print('Segment trips without sufficient time to travel before calibration:', round(j/i, 3), '    ', j, ':', i)

    l2 = df_optm_time['POSTCALI_DIFF'].tolist()
    i= len(l2)
    j = 0
    for t in l2:
        if t <= -0:
            j = j+1

    print('Segment trips without sufficient time to travel after calibration:', round(j/i, 3), '    ', j, ':', i, file=f)
    print('Segment trips without sufficient time to travel after calibration:', round(j/i, 3), '    ', j, ':', i)
    print()

#     o1 = df_optm_time['PRECALI_DIFF'].tolist()
#     i= len(o1)
#     j = 0
#     for t in o1:
#         if t <= -300 or t > 60:
#             j = j+1

#     print('Segment on time performance before calibration for the input dates:', round(1- j/i, 3), '    ', j, ':', i, file=f)
#     print('Segment on time performance before calibration for the input dates:', round(1- j/i, 3), '    ', j, ':', i)

#     o2 = df_optm_time['POSTCALI_DIFF'].tolist()
#     i= len(o2)
#     j = 0
#     for t in o2:
#         if t <= -300 or t > 60:
#             j = j+1
#     print('Segment on time performance after calibration for the input date:',  round(1-j/i, 3), '    ', j, ':', i, file=f)
#     print('Segment on time performance after calibration for the input date:',  round(1-j/i, 3), '    ', j, ':', i)

    f.close()  

    # simplify the export to service planner

    simple_df_opt = df_optm_time.drop(['Date', 'Actual Running Time','SEG','PRECALI_DIFF', 
                                       'POSTCALI_DIFF', 'Scheduled Running Time', 'ACTUAL_RUNNING_T_NO_OUTL'], axis=1)

    simple_df_opt = simple_df_opt.round({"OPTM_SCHED_TIME":0}) # around up the 

    simple_df_opt = simple_df_opt.drop_duplicates()# remove duplicates 

    simple_df_opt.to_csv('Calibration_' + file_name[:-4] + '_Brief' + '.csv')  

#     print(simple_df_opt)
    
    os.remove('df_sched_time_temporary_file.csv') # remove temporary file
    os.remove('df_optm_time_temporary_file.csv') # remove temporary file
    
    print()
    print('Done')
    print()
    print('-----------------------------------------')

In [7]:
# Test the function WEST Weekdays
def seg_order(row):
    if row['Segment'] == 'rsw - dantre':
        order = 'W1'
    elif row['Segment'] == 'dantre - btwrsh':
        order = 'W2'
    elif row['Segment'] == 'btwrsh - winsum':
        order = 'W3'
    elif row['Segment'] == 'winsum - heapar':
        order = 'W4'
    elif row['Segment'] == 'heapar - bearid':
        order = 'W5'
#    elif row['Segment'] == 'rptc - marpal':
#        order = 'E'
#     elif row['Segment'] == 'estcre - estbow': 
#         order = 'E5'
#     elif row['Segment'] == 'dantre - rsw':
#         order = 'E6'
    else:
        order = 'W'
    return order

file_name = '50 WEST Weekdays - Dec - Mar.csv'
seg_travel_cali(file_name)

Schedule Calibration: 50 WEST Weekdays - Dec - Mar

5 segments found ['rsw - dantre', 'winsum - heapar', 'dantre - btwrsh', 'heapar - bearid', 'btwrsh - winsum']

The direction has 14 different trips during the day

Total scheduled hours before calibration from the input dates: 895.58
Total scheduled hours after calibration from the input dates: 889.02

Variance (off in seconds) before calibration: 25751.03
Variance (off in seconds) after calibration: 14330.34

Segment trips have sufficient time to travel before calibration: 0.81      4094 : 5055
Segment trips have sufficient time to travel after calibration: 0.892      4509 : 5055

Segment trips without sufficient time to travel before calibration: 0.19      961 : 5055
Segment trips without sufficient time to travel after calibration: 0.108      546 : 5055


Done

-----------------------------------------


C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:111: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('Variance (off in seconds) before calibration:', round(df_optm_time.var()['PRECALI_DIFF'], 2), file=f)
C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:112: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('Variance (off in seconds) after calibration:', round(df_optm_time.var()['POSTCALI_DIFF'], 2), file=f)
C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:117: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise Typ

In [8]:
# Saturdays
def seg_order(row):
    if row['Segment'] == 'rsw - dantre':
        order = 'W1'
    elif row['Segment'] == 'dantre - btwrsh':
        order = 'W2'
    elif row['Segment'] == 'btwrsh - winsum':
        order = 'W3'
    elif row['Segment'] == 'winsum - heapar':
        order = 'W4'
    elif row['Segment'] == 'heapar - bearid':
        order = 'W5'
#    elif row['Segment'] == 'rptc - marpal':
#        order = 'E'
#     elif row['Segment'] == 'estcre - estbow': 
#         order = 'E5'
#     elif row['Segment'] == 'dantre - rsw':
#         order = 'E6'
    else:
        order = 'W'
    return order

file_name = '50 WEST Saturdays - Dec - Mar.csv'
seg_travel_cali(file_name)

Schedule Calibration: 50 WEST Saturdays - Dec - Mar

5 segments found ['rsw - dantre', 'winsum - heapar', 'dantre - btwrsh', 'heapar - bearid', 'btwrsh - winsum']

The direction has 14 different trips during the day

Total scheduled hours before calibration from the input dates: 168.28
Total scheduled hours after calibration from the input dates: 151.85

Variance (off in seconds) before calibration: 22294.27
Variance (off in seconds) after calibration: 7016.85

Segment trips have sufficient time to travel before calibration: 0.873      840 : 962
Segment trips have sufficient time to travel after calibration: 0.865      832 : 962

Segment trips without sufficient time to travel before calibration: 0.127      122 : 962
Segment trips without sufficient time to travel after calibration: 0.135      130 : 962


Done

-----------------------------------------


C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:111: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('Variance (off in seconds) before calibration:', round(df_optm_time.var()['PRECALI_DIFF'], 2), file=f)
C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:112: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('Variance (off in seconds) after calibration:', round(df_optm_time.var()['POSTCALI_DIFF'], 2), file=f)
C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:117: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise Typ

In [9]:
# Sundays
def seg_order(row):
    if row['Segment'] == 'rsw - dantre':
        order = 'W1'
    elif row['Segment'] == 'dantre - btwrsh':
        order = 'W2'
    elif row['Segment'] == 'btwrsh - winsum':
        order = 'W3'
    elif row['Segment'] == 'winsum - heapar':
        order = 'W4'
    elif row['Segment'] == 'heapar - bearid':
        order = 'W5'
#    elif row['Segment'] == 'rptc - marpal':
#        order = 'E'
#     elif row['Segment'] == 'estcre - estbow': 
#         order = 'E5'
#     elif row['Segment'] == 'dantre - rsw':
#         order = 'E6'
    else:
        order = 'W'
    return order

file_name = '50 WEST Sundays - Dec - Mar.csv'
seg_travel_cali(file_name)

Schedule Calibration: 50 WEST Sundays - Dec - Mar

5 segments found ['rsw - dantre', 'winsum - heapar', 'dantre - btwrsh', 'heapar - bearid', 'btwrsh - winsum']

The direction has 6 different trips during the day

Total scheduled hours before calibration from the input dates: 77.52
Total scheduled hours after calibration from the input dates: 67.43

Variance (off in seconds) before calibration: 21879.18
Variance (off in seconds) after calibration: 4432.7

Segment trips have sufficient time to travel before calibration: 0.902      414 : 459
Segment trips have sufficient time to travel after calibration: 0.878      403 : 459

Segment trips without sufficient time to travel before calibration: 0.098      45 : 459
Segment trips without sufficient time to travel after calibration: 0.122      56 : 459


Done

-----------------------------------------


C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:111: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('Variance (off in seconds) before calibration:', round(df_optm_time.var()['PRECALI_DIFF'], 2), file=f)
C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:112: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('Variance (off in seconds) after calibration:', round(df_optm_time.var()['POSTCALI_DIFF'], 2), file=f)
C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:117: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise Typ

In [10]:
# Test the function WEST Weekdays-Saturdays
def seg_order(row):
    if row['Segment'] == 'rsw - dantre':
        order = 'W1'
    elif row['Segment'] == 'dantre - btwrsh':
        order = 'W2'
    elif row['Segment'] == 'btwrsh - winsum':
        order = 'W3'
    elif row['Segment'] == 'winsum - heapar':
        order = 'W4'
    elif row['Segment'] == 'heapar - bearid':
        order = 'W5'
#    elif row['Segment'] == 'rptc - marpal':
#        order = 'E'
#     elif row['Segment'] == 'estcre - estbow': 
#         order = 'E5'
#     elif row['Segment'] == 'dantre - rsw':
#         order = 'E6'
    else:
        order = 'W'
    return order

file_name = '50 WEST Weekdays-Saturday - Dec - Mar.csv'
seg_travel_cali(file_name)

Schedule Calibration: 50 WEST Weekdays-Saturday - Dec - Mar

5 segments found ['rsw - dantre', 'winsum - heapar', 'dantre - btwrsh', 'heapar - bearid', 'btwrsh - winsum']

The direction has 14 different trips during the day

Total scheduled hours before calibration from the input dates: 1063.87
Total scheduled hours after calibration from the input dates: 1047.75

Variance (off in seconds) before calibration: 25331.34
Variance (off in seconds) after calibration: 13897.97

Segment trips have sufficient time to travel before calibration: 0.82      4934 : 6017
Segment trips have sufficient time to travel after calibration: 0.894      5382 : 6017

Segment trips without sufficient time to travel before calibration: 0.18      1083 : 6017
Segment trips without sufficient time to travel after calibration: 0.106      635 : 6017


Done

-----------------------------------------


C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:111: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('Variance (off in seconds) before calibration:', round(df_optm_time.var()['PRECALI_DIFF'], 2), file=f)
C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:112: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print('Variance (off in seconds) after calibration:', round(df_optm_time.var()['POSTCALI_DIFF'], 2), file=f)
C:\Users\xiw\AppData\Local\Temp\ipykernel_14636\2356051686.py:117: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise Typ